In [2]:
from model import Unet

Using TensorFlow backend.


In [3]:
unet = Unet()
model = unet.initial_model()

Initial U-Net model
Instructions for updating:
Colocations handled automatically by placer.


In [4]:
model .summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 572, 572, 1)  0                                            
__________________________________________________________________________________________________
conv1_1 (Conv2D)                (None, 570, 570, 64) 640         input_1[0][0]                    
__________________________________________________________________________________________________
conv1_2 (Conv2D)                (None, 568, 568, 64) 36928       conv1_1[0][0]                    
__________________________________________________________________________________________________
maxpooling_1 (MaxPooling2D)     (None, 284, 284, 64) 0           conv1_2[0][0]                    
__________________________________________________________________________________________________
conv2_1 (C

In [ ]:
image_path = "data/data1/aug-patch/"
mask_path = "data/data1/aug-patch/"
image_prefix = "image"
mask_prefix = "mask"
image_as_gray = True
mask_as_gray = True
image_name_arr = glob.glob(os.path.join(image_path,"*%s*.png"%image_prefix))
image_arr = []
mask_arr = []
for index,item in enumerate(image_name_arr):
    img = io.imread(item,as_gray = image_as_gray)
    img = np.reshape(img,img.shape + (1,)) if image_as_gray else img
    mask = io.imread(item.replace(image_path,mask_path).replace(image_prefix,mask_prefix),as_gray = mask_as_gray)
    mask = np.reshape(mask,mask.shape + (1,)) if mask_as_gray else mask
#     img,mask = adjustData(img,mask,flag_multi_class,num_class)
    image_arr.append(img)
    mask_arr.append(mask)
image_arr = np.array(image_arr)
mask_arr = np.array(mask_arr)

In [ ]:
np.save('image_array_model', image_arr)

In [ ]:
tmp_mask = list()
for mask in mask_arr:
    tmp_mask.append(mask[92:480, 92:480])
tmp_mask = np.array(tmp_mask)
tmp_mask.shape

In [ ]:
np.save('mask_array_model', tmp_mask)

In [5]:
import numpy as np
image_arr = np.load('image_array_model.npy')
tmp_mask = np.load('mask_array_model.npy')

In [6]:
import keras.backend as K
def iou_loss_score(true,pred):  #this can be used as a loss if you make it negative
    intersection = true * pred
    notTrue = 1 - true
    union = true + (notTrue * pred)
    return (K.sum(intersection, axis=-1) + K.epsilon()) / (K.sum(union, axis=-1) + K.epsilon())

In [7]:
model.compile(optimizer = 'Adam', loss = 'binary_crossentropy', metrics = [iou_loss_score])

In [ ]:
results = model.fit(image_arr, tmp_mask, epochs=10, verbose=1)

Instructions for updating:
Use tf.cast instead.
Epoch 1/10
